https://stackoverflow.com/questions/73095228/xarray-cannot-open-url

# this does not work on https://pangeo-clivar.vm.fedcloud.eu/jupyterhub/

Jupyterlab have 8G of RAM

In [1]:
from pyesgf.logon import LogonManager
lm = LogonManager()
lm.is_logged_on()
#hostname='esgf-node.llnl.gov'
hostname='esgf-node.ipsl.upmc.fr'
lm.logon(hostname=hostname, interactive=True, bootstrap=True, username='tinaok', password='Clivar-2022')
lm.is_logged_on()

True

In [2]:
from pyesgf.search import SearchConnection
server='https://esgf-data.dkrz.de/esg-search'
#server='https://esgf-data.llnl.gov/esg-search'
#server='https://esgf-data.ipsl.upmc.fr/esg-search'


conn = SearchConnection(server, distrib=True)

In [3]:
source_id='CMCC-CM2-HR4'
activity_id='OMIP'
experiment_id='omip2'
variable_id='vmo'
#project='CMIP6'
#variant_label='r1i1p1f2',
#data_node='esgf-data3.ceda.ac.uk'
ctx = conn.new_context(
    project='CMIP6',
    source_id=source_id,
    experiment_id=experiment_id,
    variable=variable_id,
    frequency='mon',
#    variant_label='r1i1p1f2',
#    data_node='esgf-data3.ceda.ac.uk'
)
ctx.hit_count


-------------------------------------------------------------------------------
Warning - defaulting to search with facets=*

This behavior is kept for backward-compatibility, but ESGF indexes might not
successfully perform a distributed search when this option is used, so some
results may be missing.  For full results, it is recommended to pass a list of
facets of interest when instantiating a context object.  For example,

      ctx = conn.new_context(facets='project,experiment_id')

Only the facets that you specify will be present in the facets_counts dictionary.

This warning is displayed when a distributed search is performed while using the
facets=* default, a maximum of once per context object.  To suppress this warning,
set the environment variable ESGF_PYCLIENT_NO_FACETS_STAR_WARNING to any value
or explicitly use  conn.new_context(facets='*')

-------------------------------------------------------------------------------


1

In [4]:
result = ctx.search()[0]
#result.dataset_id
files = result.file_context().search()
import xarray as xr
#gridftp=files[35].urls['GridFTP'][0][0]
url=files[35].urls['HTTPServer'][0][0]
opendap=files[35].opendap_url


-------------------------------------------------------------------------------
Warning - defaulting to search with facets=*

This behavior is kept for backward-compatibility, but ESGF indexes might not
successfully perform a distributed search when this option is used, so some
results may be missing.  For full results, it is recommended to pass a list of
facets of interest when instantiating a context object.  For example,

      ctx = conn.new_context(facets='project,experiment_id')

Only the facets that you specify will be present in the facets_counts dictionary.

This warning is displayed when a distributed search is performed while using the
facets=* default, a maximum of once per context object.  To suppress this warning,
set the environment variable ESGF_PYCLIENT_NO_FACETS_STAR_WARNING to any value
or explicitly use  conn.new_context(facets='*')

-------------------------------------------------------------------------------


https://esgf-pyclient.readthedocs.io/en/latest/notebooks/demo/subset-cmip6.html

In [5]:
ds = xr.open_dataset(opendap, chunks={'time': 120})
ds

Error:curl error: Timeout was reached
curl error details: 


OSError: [Errno -68] NetCDF: I/O failure: b'http://esgf-node2.cmcc.it/thredds/dodsC/esg_dataroot/CMIP6/OMIP/CMCC/CMCC-CM2-HR4/omip2/r1i1p1f1/Omon/vmo/gn/v20200226/vmo_Omon_CMCC-CM2-HR4_omip2_r1i1p1f1_gn_200801-201812.nc'

## Below using http access to the netcdf file itself.  It fails too on pangeo-clivar hub


In [7]:
import xarray as xr
ds_agg = xr.open_mfdataset([url], chunks={'time': 120})
ds_agg

Error:curl error: Timeout was reached
curl error details: 


OSError: [Errno -68] NetCDF: I/O failure: b'http://esgf-node2.cmcc.it/thredds/fileServer/esg_dataroot/CMIP6/OMIP/CMCC/CMCC-CM2-HR4/omip2/r1i1p1f1/Omon/vmo/gn/v20200226/vmo_Omon_CMCC-CM2-HR4_omip2_r1i1p1f1_gn_200801-201812.nc'

In [5]:
url

'http://esgf-node2.cmcc.it/thredds/fileServer/esg_dataroot/CMIP6/OMIP/CMCC/CMCC-CM2-HR4/omip2/r1i1p1f1/Omon/vmo/gn/v20200226/vmo_Omon_CMCC-CM2-HR4_omip2_r1i1p1f1_gn_200801-201812.nc'

In [6]:
!wget http://esgf-node2.cmcc.it/thredds/fileServer/esg_dataroot/CMIP6/OMIP/CMCC/CMCC-CM2-HR4/omip2/r1i1p1f1/Omon/vmo/gn/v20200226/vmo_Omon_CMCC-CM2-HR4_omip2_r1i1p1f1_gn_200801-201812.nc

--2022-10-21 10:12:11--  http://esgf-node2.cmcc.it/thredds/fileServer/esg_dataroot/CMIP6/OMIP/CMCC/CMCC-CM2-HR4/omip2/r1i1p1f1/Omon/vmo/gn/v20200226/vmo_Omon_CMCC-CM2-HR4_omip2_r1i1p1f1_gn_200801-201812.nc
Resolving esgf-node2.cmcc.it (esgf-node2.cmcc.it)... 193.204.199.223
Connecting to esgf-node2.cmcc.it (esgf-node2.cmcc.it)|193.204.199.223|:80... ^C
